In [2]:
import pandas as pd
import numpy as np

In [20]:
# выводит таблицу в таком формате: столбцы - имя gpcr, строки - BW этих gpsr(или номер остатка, для тех, у кого нет BW), а значения - минимальный номер остатка аррестина.
def table(repulsion,total):
    repulsion_R = (repulsion[repulsion.chain == 'R']
                   .merge(total[total.chain_id == 'R'], left_on=['gpcr','res'],right_on=['pdb_code','residue_number'])
                   )
    repulsion_A = (repulsion[repulsion.chain == 'A']
                   .merge(total[total.chain_id == 'A'], left_on=['arr','res'],right_on=['pdb_code','residue_number']))
    merged_repulsion = repulsion_R.merge(repulsion_A,on=['gpcr','arr','Cluster_ID'])
    total_a=total[total['chain_id']=='A'][['residue_number','residue_name']].drop_duplicates().rename(columns={'residue_number':'res_y','residue_name':'arr_residue_name'})
    b_factor_x = merged_repulsion.pivot_table(values='b_factor_x',index='res_y',columns='gpcr',aggfunc=np.min).astype('Float32').astype('str').replace('<NA>', '').reset_index().merge(total_a,on='res_y').rename(columns={'res_y':'arr_residue_number'})
    residue_name_x = merged_repulsion.pivot_table(values='residue_name_x',index='res_y',columns='gpcr',aggfunc=np.min).astype('str').replace('nan', '').reset_index().merge(total_a,on='res_y').add_suffix('_name').rename(columns={'res_y_name':'arr_residue_number'}).drop(columns=['arr_residue_name_name'])
    dist_x = merged_repulsion.pivot_table(values='dist_x',index='res_y',columns='gpcr',aggfunc=np.min).astype('Float32').round(2).astype('str').replace('<NA>', '').reset_index().merge(total_a,on='res_y').add_suffix('_dist').rename(columns={'res_y_dist':'arr_residue_number'}).drop(columns=['arr_residue_name_dist'])
    merged = b_factor_x .merge(residue_name_x, on = 'arr_residue_number') .merge(dist_x, on = 'arr_residue_number')
    return  merged.reindex(sorted(merged.columns), axis=1).drop_duplicates(subset='arr_residue_number').reset_index(drop=True)

In [23]:
def table_simple(repulsion,total):
    repulsion_R = (repulsion[repulsion.chain == 'R']
                   .merge(total[total.chain_id == 'R'], left_on=['gpcr','res'],right_on=['pdb_code','residue_number'])
                   )
    repulsion_A = (repulsion[repulsion.chain == 'A']
                   .merge(total[total.chain_id == 'A'], left_on=['arr','res'],right_on=['pdb_code','residue_number']))
    merged_repulsion = repulsion_R.merge(repulsion_A,on=['gpcr','arr','Cluster_ID'])
    total_a=total[total['chain_id']=='A'][['residue_number','residue_name']].drop_duplicates().rename(columns={'residue_number':'res_y','residue_name':'arr_residue_name'})
    b_factor_x = merged_repulsion.pivot_table(values='b_factor_x',index='res_y',columns='gpcr',aggfunc=np.min).astype('Float32').astype('str').replace('<NA>', '').reset_index().merge(total_a,on='res_y').rename(columns={'res_y':'arr_residue_number'})
    residue_name_x = merged_repulsion.pivot_table(values='residue_name_x',index='res_y',columns='gpcr',aggfunc=np.min).astype('str').replace('nan', '').reset_index().merge(total_a,on='res_y').add_suffix('_name').rename(columns={'res_y_name':'arr_residue_number'}).drop(columns=['arr_residue_name_name'])
    merged = b_factor_x .merge(residue_name_x, on = 'arr_residue_number')
    return  merged.reindex(sorted(merged.columns), axis=1).drop_duplicates(subset='arr_residue_number').reset_index(drop=True)

In [10]:
AT14 = ['4YAY','4ZUD']
AT16 = ['6DO1','6OS0','6OS1','6OS2']
AT1 = AT14 + AT16
arr = ['6TKO','6U1N','6UP7','6PWC']

In [11]:
hyd_aci = pd.read_csv('/home/kestr/AT1/data/hydrogen_contacts.csv')
repulsion = pd.read_csv('/home/kestr/AT1/data/charge_repulsion.csv')
other_contacts = pd.read_csv('/home/kestr/AT1/data/contacts_ion_and_hydrophobic.csv')
total = pd.read_csv('/home/kestr/AT1/data/model_res_dict.csv')
hydrophob = other_contacts[other_contacts.type=='PH'].drop(columns=['type'])
ionic = other_contacts[other_contacts.type=='IO'].drop(columns=['type'])

In [31]:
total.query('b_factor == 3.54')

,chain_id,residue_number,residue_name,b_factor,pdb_code
118,R,130,ILE,3.54,4YAY
406,R,130,ILE,3.54,4ZUD
680,R,130,ILE,3.54,6DO1
980,R,130,ILE,3.54,6OS0
1278,R,130,ILE,3.54,6OS1
1580,R,130,ILE,3.54,6OS2
2208,R,143,ILE,3.54,6TKO
2851,R,125,VAL,3.54,6U1N
3479,R,170,ILE,3.54,6UP7
4152,R,170,ILE,3.54,6PWC


In [24]:
# водородные связи
table(hyd_aci,total).to_csv('/home/kestr/AT1/data/table_contacts/HY.csv',index=False)
table_simple(hyd_aci,total)

,4YAY,4YAY_name,4ZUD,4ZUD_name,6DO1,6DO1_name,6OS0,6OS0_name,6OS1,6OS1_name,6OS2,6OS2_name,6TKO,6TKO_name,6U1N,6U1N_name,6UP7,6UP7_name,arr_residue_name,arr_residue_number
0,,,,,34.51,MET,34.51,MET,34.5,PRO,34.5,PRO,34.54,GLN,,,,,TYR,63
1,,,,,34.54,ARG,34.54,ARG,34.54,ARG,34.54,ARG,34.54,GLN,,,2.36,GLN,ARG,65
2,2.37,THR,2.37,THR,2.37,ARG,2.37,THR,2.37,THR,2.37,THR,2.37,THR,,,,,GLU,66
3,6.3,ASN,3.5,ARG,,,3.5,ARG,,,,,12.49,ARG,,,,,ASP,67
4,6.3,ASN,3.5,ARG,7.56,LEU,7.56,LEU,7.56,LEU,7.56,LEU,,,,,,,CYS,68
5,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,,,,,ASP,69
6,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,,,,,,,,,,,VAL,70
7,3.5,ARG,6.3,ASN,,,3.5,ARG,,,8.48,LYS,,,,,6.29,ALA,LEU,71
8,6.31,ASP,6.31,ASP,,,,,,,,,,,,,,,GLY,72
9,6.31,ASP,6.3,ASN,,,,,,,,,,,,,,,LEU,73


In [25]:
# стерическое отталкивание
table(repulsion,total).to_csv('/home/kestr/AT1/data/table_contacts/RE.csv',index=False)
table(repulsion,total)

,4YAY,4YAY_dist,4YAY_name,4ZUD,4ZUD_dist,4ZUD_name,6DO1,6DO1_dist,6DO1_name,6OS0,6OS0_dist,6OS0_name,6OS1,6OS1_dist,6OS1_name,6OS2,6OS2_dist,6OS2_name,arr_residue_name,arr_residue_number
0,,,,,,,34.54,1.13,ARG,34.54,0.67,ARG,34.54,0.85,ARG,34.54,0.72,ARG,ARG,65
1,6.3,1.73,ASN,,,,,,,,,,,,,,,,ASP,67
2,6.31,1.5,ASP,6.3,1.23,ASN,,,,,,,,,,,,,CYS,68
3,6.3,0.51,ASN,6.3,0.85,ASN,,,,7.56,1.79,LEU,,,,,,,ASP,69
4,3.5,1.12,ARG,3.5,1.12,ARG,,,,3.5,1.17,ARG,3.5,1.63,ARG,3.5,0.95,ARG,VAL,70
5,6.3,0.96,ASN,6.3,0.91,ASN,,,,,,,,,,,,,LEU,71
6,6.31,0.37,ASP,6.31,0.73,ASP,,,,,,,,,,,,,GLY,72
7,6.31,1.93,ASP,6.31,1.17,ASP,,,,,,,,,,,,,LEU,73
8,34.53,1.93,SER,,,,,,,,,,,,,,,,THR,246
9,34.54,1.12,ARG,,,,,,,,,,,,,,,,ARG,285


In [26]:
# гидрофобное притяжение
table_simple(hydrophob,total).to_csv('/home/kestr/AT1/data/table_contacts/PH.csv',index=False)
table_simple(hydrophob,total)

,4YAY,4YAY_name,4ZUD,4ZUD_name,6DO1,6DO1_name,6OS0,6OS0_name,6OS1,6OS1_name,...,6PWC,6PWC_name,6TKO,6TKO_name,6U1N,6U1N_name,6UP7,6UP7_name,arr_residue_name,arr_residue_number
0,3.54,ILE,3.54,ILE,3.54,ILE,3.54,ILE,3.55,ILE,...,,,,,,,3.54,ILE,CYS,68
1,3.54,ILE,3.54,ILE,3.54,ILE,2.43,ILE,2.43,ILE,...,,,1.4,ALA,,,3.54,ILE,VAL,70
2,3.54,ILE,3.54,ILE,3.54,ILE,2.43,ILE,2.43,ILE,...,,,3.23,ALA,,,3.54,ILE,LEU,71
3,3.54,ILE,3.54,ILE,3.54,ILE,2.43,ILE,2.43,ILE,...,5.69,VAL,3.23,ALA,,,,,LEU,73
4,,,,,,,,,,,...,,,,,34.51,LEU,,,LEU,129
5,,,,,,,,,,,...,,,,,34.51,LEU,,,CYS,140
6,,,,,,,,,,,...,,,,,34.51,LEU,,,ILE,241


In [27]:
# ионное притяжение
table_simple(ionic,total).to_csv('/home/kestr/AT1/data/table_contacts/IO.csv',index=False)
table_simple(ionic,total)

,4ZUD,4ZUD_name,6OS0,6OS0_name,6OS1,6OS1_name,6OS2,6OS2_name,6TKO,6TKO_name,arr_residue_name,arr_residue_number
0,,,,,,,,,12.49,ARG,ASP,67
1,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,3.5,ARG,ASP,69
